In [1]:
import os
import matplotlib.pyplot as plt
import argparse
import numpy as np

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Input, Embedding, LSTM, Dense, Conv2D, Flatten, Reshape, Add, Concatenate, MaxPool2D
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam
from keras import backend as K

from model.unet import unet, unet_small, dice_coef, normalized_iou_coef, hard_normalized_hard_iou_coef
from model.fcn import fcn_8s
from model.pspnet import pspnet50
from dataset_parser import generator_miscl, generator

from scripts import *

%load_ext autoreload
%autoreload 2

print('GPUs available:', K.tensorflow_backend._get_available_gpus())

CLEAN_N = 500
TRAIN_N = 2975
VAL_N = 500

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.






GPUs available: ['/job:localhost/replica:0/task:0/device:GPU:0']


# Clean Model

In [2]:
# TRAIN_BATCH = int(4)
# VAL_BATCH = int(4)
# lr_init = 1e-4
# lr_decay = 5e-4
# vgg_path = None
# TRAIN_N = 2975
# VAL_N = 500

# model_name = "modular_unet_clean_benchmark_other_metrics.h5"

# labels = ['background', 'car', 'road']  # Use only 3 classes.
# ignore_pedestrians=not('person' in labels)

# clean_model = unet(input_shape=(256, 512, 3), num_classes=len(labels),
#                     lr_init=lr_init, lr_decay=lr_decay, vgg_weight_path=vgg_path)

In [3]:
# checkpoint = ModelCheckpoint(filepath=model_name,
#                              monitor='val_dice_coef',
#                              mode='max',
#                              save_best_only=True,
#                              save_weights_only=True)

In [4]:
# history = clean_model.fit_generator(generator.data_generator('dataset_parser/data.h5', TRAIN_BATCH, 'train', ignore_pedestrians=ignore_pedestrians),
#                                     steps_per_epoch= TRAIN_N // TRAIN_BATCH,
#                                     validation_data=generator.data_generator('dataset_parser/data.h5', VAL_BATCH, 'val', ignore_pedestrians=ignore_pedestrians),
#                                     validation_steps= VAL_N // VAL_BATCH,
#                                     callbacks=[checkpoint],
#                                     epochs=5, verbose=1)

In [5]:
# clean_model.load_weights(model_name)

# Just Gold Data

In [6]:
TRAIN_BATCH = int(4)
VAL_BATCH = int(4)
lr_init = 1e-4
lr_decay = 5e-4
vgg_path = None
TRAIN_N = 2975
VAL_N = 500

model_name = "modular_unet_gold_benchmark_other_metrics.h5"

labels = ['background', 'car', 'road']  # Use only 3 classes.
ignore_pedestrians=not('person' in labels)

gold_model = unet(input_shape=(256, 512, 3), num_classes=len(labels),
                    lr_init=lr_init, lr_decay=lr_decay, vgg_weight_path=vgg_path)









Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:



In [7]:
checkpoint = ModelCheckpoint(filepath=model_name,
                             monitor='val_dice_coef',
                             mode='max',
                             save_best_only=True,
                             save_weights_only=True)

In [8]:
fine_generator = generator.data_generator('dataset_parser/data.h5', TRAIN_BATCH, 'train', ignore_pedestrians=ignore_pedestrians)

history = gold_model.fit_generator(fine_generator,
                                   steps_per_epoch= CLEAN_N // TRAIN_BATCH,
                                   validation_data=generator.data_generator('dataset_parser/data.h5', VAL_BATCH, 'val', ignore_pedestrians=ignore_pedestrians),
                                   validation_steps= VAL_N // VAL_BATCH,
                                   callbacks=[checkpoint],
                                   epochs=5, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
5002975
2975

500


125/125 [==============================] - 61s 490ms/step - loss: 0.4581 - dice_coef: 0.7258 - normalized_iou_coef: 0.4350 - hard_normalized_hard_iou_coef: 0.5145 - val_loss: 0.4624 - val_dice_coef: 0.8004 - val_normalized_iou_coef: 0.5019 - val_hard_normalized_hard_iou_coef: 0.5424
Epoch 2/5
125/125 [==============================] - 54s 435ms/step - loss: 0.3004 - dice_coef: 0.8203 - normalized_iou_coef: 0.5507 - hard_normalized_hard_iou_coef: 0.6506 - val_loss: 0.3480 - val_dice_coef: 0.8369 - val_normalized_iou_coef: 0.5775 - val_hard_normalized_hard_iou_coef: 0.6348
Epoch 3/5
125/125 [==============================] - 55s 442ms/step - loss: 0.2613 - dice_coef: 0.8495 - normalized_iou_coef: 0.6318 - hard_normalized_hard_iou_coef: 0.7208 - val_loss: 0.3224 - val_dice_coef: 0.8231 - val_normalized_iou_coef: 0.5836 - val_hard_normalized_hard_iou_coef: 0.6542
Ep

# Pseudolabeling

In [9]:
TRAIN_BATCH = int(4)
VAL_BATCH = int(4)
lr_init = 1e-4
lr_decay = 5e-4
vgg_path = None

model_name = "modular_unet_pseu_benchmark_other_metrics_miscl50.h5"

labels = ['background', 'car', 'road']  # Use only 3 classes.
ignore_pedestrians=not('person' in labels)

pseu_model = unet(input_shape=(256, 512, 3), num_classes=len(labels),
                    lr_init=lr_init, lr_decay=lr_decay, vgg_weight_path=vgg_path)

In [10]:
checkpoint = ModelCheckpoint(filepath=model_name,
                             monitor='val_dice_coef',
                             mode='max',
                             save_best_only=True,
                             save_weights_only=True)

In [11]:
n_epochs = 3

for e in range(n_epochs):
    miscl_generator = generator_miscl.data_generator('dataset_parser/data.h5', TRAIN_BATCH, 'train', ignore_pedestrians=ignore_pedestrians, misclassified_frac=0.75)
    
    for i in range(TRAIN_N // TRAIN_BATCH):
        x, _ = next(miscl_generator)
        y_pred = gold_model.predict(x)
        metrics = pseu_model.train_on_batch(x, y_pred)  # x and y
        print(i, metrics)
    
#     metrics = ec_model.evaluate(y_val_coarse_batch, y_val_diff_batch)  # y only
    metrics = pseu_model.evaluate_generator(generator.data_generator('dataset_parser/data.h5', VAL_BATCH, 'val', ignore_pedestrians=ignore_pedestrians),
                              steps=VAL_N // VAL_BATCH)  # x and y 
    print('\nEpoch {}, batch {}, Validation metrics (loss, dice_coef, iou, hard_iou): {}'.format(e, i, metrics))

2975
2975
0 [1.2951927, 0.31774426, 0.15595505, 0.13630341572453578]
1 [1.172651, 0.35128888, 0.17157346, 0.19708778832827548]
2 [1.0753746, 0.38295755, 0.19062117, 0.2538327637998883]
3 [0.98194784, 0.41221872, 0.2097463, 0.31707882516678865]
4 [0.9087392, 0.44900003, 0.23194742, 0.35352434314319303]
5 [0.8912302, 0.46478024, 0.23666307, 0.3501153139573967]
6 [0.7811033, 0.5119847, 0.27240252, 0.4166431857845712]
7 [0.7473309, 0.531157, 0.2879245, 0.4362137528678971]
8 [0.7874252, 0.5246181, 0.27715334, 0.3909336709177781]
9 [0.73160225, 0.5507886, 0.294482, 0.4244815586506654]
10 [0.7548401, 0.5379761, 0.2807083, 0.3902587021673208]
11 [0.6831337, 0.57821125, 0.30815965, 0.43382108697511335]
12 [0.6296258, 0.6039884, 0.32674783, 0.46021021872086876]
13 [0.6809969, 0.59177995, 0.31249458, 0.42206513402229706]
14 [0.64247966, 0.60223556, 0.32170227, 0.46658038721769035]
15 [0.60484576, 0.6150248, 0.32640383, 0.47084099882073477]
16 [0.6110352, 0.61564654, 0.326702, 0.46875675793700855]

# Coarse Data

In [12]:
TRAIN_BATCH = int(4)
VAL_BATCH = int(4)
lr_init = 1e-4
lr_decay = 5e-4
vgg_path = None
TRAIN_N = 2975
VAL_N = 500

model_name = "modular_unet_misc50_benchmark_other_metrics.h5"

labels = ['background', 'car', 'road']  # Use only 3 classes.
ignore_pedestrians=not('person' in labels)

coarse_model = unet(input_shape=(256, 512, 3), num_classes=len(labels),
                    lr_init=lr_init, lr_decay=lr_decay, vgg_weight_path=vgg_path)

In [13]:
checkpoint = ModelCheckpoint(filepath=model_name,
                             monitor='val_dice_coef',
                             mode='max',                             
                             save_best_only=True,
                             save_weights_only=True)

In [14]:
miscl_generator = generator_miscl.data_generator('dataset_parser/data.h5', TRAIN_BATCH, 'train', ignore_pedestrians=ignore_pedestrians, misclassified_frac=0.75)

In [15]:
history = coarse_model.fit_generator(miscl_generator,
                                     steps_per_epoch= TRAIN_N // TRAIN_BATCH,
                                     validation_data=generator.data_generator('dataset_parser/data.h5', VAL_BATCH, 'val', ignore_pedestrians=ignore_pedestrians),
                                     validation_steps=VAL_N // VAL_BATCH,
                                     callbacks=[checkpoint],
                                     epochs=5, verbose=1)

Epoch 1/5
500
500
2975
2975
743/743 [==============================] - 263s 354ms/step - loss: 0.3053 - dice_coef: 0.8239 - normalized_iou_coef: 0.4843 - hard_normalized_hard_iou_coef: nan - val_loss: 0.4536 - val_dice_coef: 0.7803 - val_normalized_iou_coef: 0.4615 - val_hard_normalized_hard_iou_coef: 0.4758
Epoch 2/5
743/743 [==============================] - 257s 347ms/step - loss: 0.2124 - dice_coef: 0.8806 - normalized_iou_coef: 0.5416 - hard_normalized_hard_iou_coef: nan - val_loss: 0.3099 - val_dice_coef: 0.8554 - val_normalized_iou_coef: 0.5464 - val_hard_normalized_hard_iou_coef: 0.5334
Epoch 3/5
743/743 [==============================] - 257s 347ms/step - loss: 0.1924 - dice_coef: 0.8930 - normalized_iou_coef: 0.5632 - hard_normalized_hard_iou_coef: nan - val_loss: 0.3302 - val_dice_coef: 0.8434 - val_normalized_iou_coef: 0.5508 - val_hard_normalized_hard_iou_coef: 0.5197
Epoch 4/5
743/743 [==============================] - 257s 346ms/step - loss: 0.1742 - dice_coef: 0.9026 - 

# Apply Error Correction Networks (Full)

In [16]:
from keras.layers import Input, Embedding, LSTM, Dense, Conv2D, Flatten, Reshape, Add, Concatenate, MaxPool2D, Lambda
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam
from keras import backend as K

input_window_size = 64
output_window_size = 1

# ec_model = get_ec_model(input_window_size, output_window_size, dice_coef=dice_coef)

lr_init = 1e-3
lr_decay = 5e-4

inp_y = Input(shape=(input_window_size, input_window_size, 3))
inp_x = Input(shape=(input_window_size, input_window_size, 3))

h = Concatenate()([inp_x, inp_y])
h = Conv2D(8, (4, 4), padding='same', activation='relu')(h)
h = MaxPool2D((2, 2), padding='same')(h)
h = Conv2D(8, (4, 4), padding='same', activation='relu')(h)
h = MaxPool2D((2, 2), padding='same')(h)
h = Conv2D(16, (4, 4), padding='same', activation='relu')(h)
h = Flatten()(h)
h = Dense(40, activation='relu')(h)
h = Dense(3, activation='softmax')(h)
h = Reshape((1, 1, 3))(h)
ec_model = Model(inputs=[inp_x, inp_y], outputs=h)

ec_model.summary()
metrics=[dice_coef, normalized_iou_coef, hard_normalized_hard_iou_coef]
ec_model.compile(optimizer=Adam(lr=lr_init, decay=lr_decay),
                 loss='categorical_crossentropy', metrics=metrics)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
concatenate_13 (Concatenate)    (None, 64, 64, 6)    0           input_5[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 64, 64, 8)    776         concatenate_13[0][0]             
__________

In [17]:
n_epochs = 3
batch_size = 8
skip = 12
N_CLEAN = 500
best_val = 0

val_generator = generator.data_generator('dataset_parser/data.h5', 32, 'val', ignore_pedestrians=True)

x_val, y_val = next(val_generator)
y_val_coarse = coarse_model.predict(x_val)
x_val_batch, y_val_coarse_batch, y_val_diff_batch = get_ec_batch(
    x_val, y_val, y_val_coarse, input_window_size=input_window_size,
    output_window_size=output_window_size, skip=skip, diff=False)

for e in range(n_epochs):
    fine_generator = generator.data_generator('dataset_parser/data.h5', batch_size, 'train', 
                                              ignore_pedestrians=ignore_pedestrians)
    for i in range(N_CLEAN // batch_size):
        x, y_fine = next(fine_generator)
        y_coarse = coarse_model.predict(x)
        x_batch, y_coarse_batch, y_diff_batch = get_ec_batch(x, y_fine, y_coarse, input_window_size=input_window_size,
                                                             output_window_size=output_window_size, skip=skip, diff=False,
                                                             balanced=True)
        
#         metrics = ec_model.train_on_batch(y_coarse_batch, y_diff_batch)
        metrics = ec_model.train_on_batch([x_batch, y_coarse_batch], y_diff_batch)  # x and y
        print(i, metrics)
    
#     metrics = ec_model.evaluate(y_val_coarse_batch, y_val_diff_batch)  # y only
    metrics = ec_model.evaluate([x_val_batch, y_val_coarse_batch], y_val_diff_batch)  # x and y 
    print('\nEpoch {}, batch {}, Validation metrics (loss, dice_coef, iou, hard_iou): {}'.format(e, i, metrics))
    if metrics[-1] > best_val:
        best_val = metrics[-1]
        ec_model.save('best-ec-model-5.h5')
        print('Model saved')

500
500
2975
2975
0 [1.156331, 0.3355385, 0.1897732, 0.22579045533846998]
1 [0.94132, 0.41835922, 0.22558014, 0.3089135791324571]
2 [0.9782071, 0.43643335, 0.23788571, 0.40443912812333865]
3 [0.713953, 0.519236, 0.27366066, 0.46072215251638093]
4 [0.8352566, 0.5160059, 0.2906072, 0.42400841437918846]
5 [0.7065705, 0.57745814, 0.32746005, 0.43927812679048595]
6 [0.5597973, 0.6545451, 0.38425046, 0.47307326155674884]
7 [0.5593126, 0.6812492, 0.41263008, 0.4682229537303805]
8 [0.6245425, 0.65806675, 0.434558, 0.4707155797101449]
9 [0.5302482, 0.7036397, 0.47518533, 0.4859440213105611]
10 [0.51681316, 0.7267844, 0.47241777, 0.4754811344644213]
11 [0.55414236, 0.69690937, 0.448111, 0.47296613230271334]
12 [0.55401826, 0.7055476, 0.47184795, 0.48148632407392516]
13 [0.6981344, 0.6982988, 0.44924673, 0.46035837911835087]
14 [0.4886224, 0.71959037, 0.477942, 0.49918246968845176]
15 [0.5806784, 0.7369867, 0.4525962, 0.5252993989083572]
16 [0.5593978, 0.71557236, 0.47161907, 0.4810671103979216]


In [18]:
y_p = ec_model.predict([x_val_batch, y_val_coarse_batch])

In [19]:
np_hard_normalized_hard_iou_coef(y_val_diff_batch, y_p)

0.7661751181407229

# Apply Error Correction Networks (Y only)

In [34]:
def get_ec_model():
    input_window_size = 64
    output_window_size = 1

    # ec_model = get_ec_model(input_window_size, output_window_size, dice_coef=dice_coef)

    lr_init = 1e-3
    lr_decay = 5e-4

    inp_y = Input(shape=(input_window_size, input_window_size, 3))
    inp_x = Input(shape=(input_window_size, input_window_size, 3))

    h = Concatenate()([inp_x, inp_y])
    h = Conv2D(8, (4, 4), padding='same', activation='relu')(h)
    h = MaxPool2D((2, 2), padding='same')(h)
    h = Conv2D(8, (4, 4), padding='same', activation='relu')(h)
    h = MaxPool2D((2, 2), padding='same')(h)
    h = Conv2D(16, (4, 4), padding='same', activation='relu')(h)
    h = Flatten()(h)
    h = Dense(40, activation='relu')(h)
    h = Dense(3, activation='softmax')(h)
    h = Reshape((1, 1, 3))(h)
    ec_model = Model(inputs=[inp_x, inp_y], outputs=h)

    ec_model.summary()
    metrics=[dice_coef, normalized_iou_coef, hard_normalized_hard_iou_coef]
    ec_model.compile(optimizer=Adam(lr=lr_init, decay=lr_decay),
                     loss='categorical_crossentropy', metrics=metrics) 
    return ec_model

In [35]:
ec_model = get_ec_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
concatenate_16 (Concatenate)    (None, 64, 64, 6)    0           input_11[0][0]                   
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 64, 64, 8)    776         concatenate_16[0][0]             
__________

In [36]:
n_epochs = 3
batch_size = 8
skip = 12
N_CLEAN = 500
best_val = 0

val_generator = generator.data_generator('dataset_parser/data.h5', 32, 'val', ignore_pedestrians=True)

x_val, y_val = next(val_generator)
y_val_coarse = coarse_model.predict(x_val)
x_val_batch, y_val_coarse_batch, y_val_diff_batch = get_ec_batch(
    x_val, y_val, y_val_coarse, input_window_size=input_window_size,
    output_window_size=output_window_size, skip=skip, diff=False)

for e in range(n_epochs):
    fine_generator = generator.data_generator('dataset_parser/data.h5', batch_size, 'train', 
                                              ignore_pedestrians=ignore_pedestrians)
    for i in range(N_CLEAN // batch_size):
        x, y_fine = next(fine_generator)
        y_coarse = coarse_model.predict(x)
        x_batch, y_coarse_batch, y_diff_batch = get_ec_batch(x, y_fine, y_coarse, input_window_size=input_window_size,
                                                             output_window_size=output_window_size, skip=skip, diff=False,
                                                             balanced=True)
        x_batch = np.random.random(x_batch.shape)
#         metrics = ec_model.train_on_batch(y_coarse_batch, y_diff_batch)
        metrics = ec_model.train_on_batch([x_batch, y_coarse_batch], y_diff_batch)  # x and y
        print(i, metrics)
    
#     metrics = ec_model.evaluate(y_val_coarse_batch, y_val_diff_batch)  # y only
    metrics = ec_model.evaluate([x_val_batch, y_val_coarse_batch], y_val_diff_batch)  # x and y 
    print('\nEpoch {}, batch {}, Validation metrics (loss, dice_coef, iou, hard_iou): {}'.format(e, i, metrics))
    if metrics[-1] > best_val:
        best_val = metrics[-1]
        ec_model.save('best-ec-model-5.h5')
        print('Model saved')

500
500
2975
2975
0 [1.0729073, 0.34355095, 0.19596145, 0.2077162918549874]
1 [0.95711124, 0.41261312, 0.22881588, 0.2077591646913462]
2 [0.9379883, 0.46994194, 0.26828325, 0.4127130875224661]
3 [0.6534528, 0.54889935, 0.30192614, 0.4600078605907621]
4 [0.6696214, 0.61693656, 0.3591296, 0.4407243635731211]
5 [0.62624824, 0.692787, 0.39463374, 0.4146664277171197]
6 [0.52085704, 0.72362167, 0.44390076, 0.46623048911986703]
7 [0.4948848, 0.74443686, 0.4727965, 0.4797441904183477]
8 [0.60397416, 0.7092926, 0.48425683, 0.48790365986537765]
9 [0.5662575, 0.70095986, 0.47729287, 0.5061204684102191]
10 [0.6116365, 0.67674613, 0.44575286, 0.5100346096357633]
11 [0.615294, 0.6613297, 0.42181742, 0.512511689297146]
12 [0.5595483, 0.6795629, 0.45756483, 0.562444952777787]
13 [0.60127264, 0.6725861, 0.448357, 0.5227847366032229]
14 [0.5009852, 0.6945551, 0.46268862, 0.5776678234962408]
15 [0.5842137, 0.7017104, 0.43264383, 0.5148802263995205]
16 [0.58217645, 0.6940796, 0.44904602, 0.497778350448966

In [39]:
y_p = ec_model.predict([x_val_batch, y_val_coarse_batch])

In [40]:
np_hard_normalized_hard_iou_coef(y_val_diff_batch, y_p)

0.3657540503620878

# Apply Error Correction Networks (X only)

In [ ]:
ec_model = get_ec_model()

In [ ]:
n_epochs = 3
batch_size = 8
skip = 12
N_CLEAN = 500
best_val = 0

val_generator = generator.data_generator('dataset_parser/data.h5', 32, 'val', ignore_pedestrians=True)

x_val, y_val = next(val_generator)
y_val_coarse = coarse_model.predict(x_val)
x_val_batch, y_val_coarse_batch, y_val_diff_batch = get_ec_batch(
    x_val, y_val, y_val_coarse, input_window_size=input_window_size,
    output_window_size=output_window_size, skip=skip, diff=False)

for e in range(n_epochs):
    fine_generator = generator.data_generator('dataset_parser/data.h5', batch_size, 'train', 
                                              ignore_pedestrians=ignore_pedestrians)
    for i in range(N_CLEAN // batch_size):
        x, y_fine = next(fine_generator)
        y_coarse = coarse_model.predict(x)
        x_batch, y_coarse_batch, y_diff_batch = get_ec_batch(x, y_fine, y_coarse, input_window_size=input_window_size,
                                                             output_window_size=output_window_size, skip=skip, diff=False,
                                                             balanced=True)
        y_coarse_batch = np.random.random(y_coarse_batch.shape)
#         metrics = ec_model.train_on_batch(y_coarse_batch, y_diff_batch)
        metrics = ec_model.train_on_batch([x_batch, y_coarse_batch], y_diff_batch)  # x and y
        print(i, metrics)
    
#     metrics = ec_model.evaluate(y_val_coarse_batch, y_val_diff_batch)  # y only
    metrics = ec_model.evaluate([x_val_batch, y_val_coarse_batch], y_val_diff_batch)  # x and y 
    print('\nEpoch {}, batch {}, Validation metrics (loss, dice_coef, iou, hard_iou): {}'.format(e, i, metrics))
    if metrics[-1] > best_val:
        best_val = metrics[-1]
        ec_model.save('best-ec-model-5.h5')
        print('Model saved')

In [32]:
y_p = ec_model.predict([x_val_batch, y_val_coarse_batch])

In [33]:
np_hard_normalized_hard_iou_coef(y_val_diff_batch, y_p)

0.19444097182357953